In [ ]:
# ติดตั้ง AutoGluon และ library สำหรับดาวน์โหลดข้อมูลจาก Kaggle
# ขั้นตอนนี้อาจใช้เวลา 2-3 นาที
%pip install --upgrade "autogluon[all]" opendatasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━

In [1]:
import opendatasets as od
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor

# ระบุ URL ของชุดข้อมูลบน Kaggle
dataset_url = 'https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques'

# ดาวน์โหลดข้อมูล (ระบบจะถามหา Kaggle Username และ Key)
od.download(dataset_url)

# กำหนด Path ของไฟล์ข้อมูล
data_dir = './house-prices-advanced-regression-techniques'

# โหลดข้อมูล train และ test ด้วย pandas
train_data = pd.read_csv(f'{data_dir}/train.csv')
test_data = pd.read_csv(f'{data_dir}/test.csv')

# แสดงขนาดและตัวอย่างข้อมูลเพื่อตรวจสอบความถูกต้อง
print(f"ขนาดข้อมูล Train: {train_data.shape}")
print(f"ขนาดข้อมูล Test:  {test_data.shape}")
print("\nตัวอย่างข้อมูล Train 5 แถวแรก:")
train_data.head()


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading house-prices-advanced-regression-techniques.zip to .\house-prices-advanced-regression-techniques


100%|██████████| 199k/199k [00:00<00:00, 199MB/s]


Extracting archive .\house-prices-advanced-regression-techniques/house-prices-advanced-regression-techniques.zip to .\house-prices-advanced-regression-techniques
ขนาดข้อมูล Train: (1460, 81)
ขนาดข้อมูล Test:  (1459, 80)

ตัวอย่างข้อมูล Train 5 แถวแรก:


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [2]:
# เก็บ Id ของ test_data ไว้สำหรับสร้างไฟล์ submission
test_ids = test_data['Id']

# แปลง DataFrame ของ pandas เป็น TabularDataset ซึ่งเป็นฟอร์แมตของ AutoGluon
# ในกรณีนี้ แค่ส่ง DataFrame เข้าไปตรงๆ ก็ได้
train_data_ag = TabularDataset(train_data)
test_data_ag = TabularDataset(test_data)

# เราไม่จำเป็นต้องลบคอลัมน์ Id ออกเอง AutoGluon ฉลาดพอที่จะไม่นำมันไปใช้ในการเรียนรู้
# หากชื่อคอลัมน์ไม่ตรงกับ label


In [4]:
# กำหนดคอลัมน์เป้าหมาย
label = 'SalePrice'

# กำหนดเวลาในการฝึก (หน่วยเป็นวินาที), 600 วินาที = 10 นาที
time_limit = 600

# สร้าง Predictor
predictor = TabularPredictor(
    label=label,
    eval_metric='root_mean_squared_error',
    path='ag_models_house_prices' # โฟลเดอร์สำหรับเซฟโมเดล
)

# เริ่มฝึกโมเดล
# AutoGluon จะจัดการข้อมูลที่ซับซ้อนและมี Missing Value จำนวนมากให้เอง
predictor.fit(
    train_data_ag,
    time_limit=time_limit,
    presets='best_quality'
)


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.12.0
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          16
Memory Avail:       1.44 GB / 15.71 GB (9.1%)
Disk Space Avail:   4.15 GB / 475.67 GB (0.9%)
	We recommend a minimum available disk space of 10 GB, and large datasets may require more.
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then

(_ray_fit pid=34228) [1000]	valid_set's rmse: 31709.8
(_ray_fit pid=34228) [2000]	valid_set's rmse: 31247.2
(_ray_fit pid=34228) [3000]	valid_set's rmse: 31101.6
(_ray_fit pid=34228) [4000]	valid_set's rmse: 31065.2
(_ray_fit pid=34228) [5000]	valid_set's rmse: 31054.2
(_ray_fit pid=34228) [6000]	valid_set's rmse: 31052.2
(_ray_fit pid=34228) [7000]	valid_set's rmse: 31051.8
(_ray_fit pid=34228) [8000]	valid_set's rmse: 31051.7


KeyboardInterrupt: 

(_ray_fit pid=34228) [9000]	valid_set's rmse: 31051.6


In [5]:
# แสดง Leaderboard ของโมเดลทั้งหมด
# สำหรับ RMSE ค่ายิ่งน้อยยิ่งดี
predictor.leaderboard(train_data_ag, silent=True)


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,KNeighborsDist_BAG_L1,-0.000000,-46136.595573,root_mean_squared_error,0.020691,0.031923,0.012974,0.020691,0.031923,0.012974,1,True,2
1,LightGBMXT_BAG_L1,-10006.991363,-25411.711726,root_mean_squared_error,10.961166,2.019149,64.795738,10.961166,2.019149,64.795738,1,True,3
2,RandomForestMSE_BAG_L1,-10718.044644,-29199.854464,root_mean_squared_error,0.213617,0.440006,10.480198,0.213617,0.440006,10.480198,1,True,5
3,ExtraTreesMSE_BAG_L1,-10797.071349,-29473.045014,root_mean_squared_error,0.199049,0.422204,5.184081,0.199049,0.422204,5.184081,1,True,7
4,WeightedEnsemble_L3,-12167.380897,-24803.958666,root_mean_squared_error,15.113260,4.980953,303.435495,0.003811,0.000747,0.026243,3,True,11
5,WeightedEnsemble_L2,-12337.202464,-24803.867937,root_mean_squared_error,11.479352,3.003763,229.438311,0.003531,0.000854,0.028485,2,True,8
6,LightGBMXT_BAG_L2,-13011.354046,-26373.852912,root_mean_squared_error,15.109448,4.980206,303.409252,3.178286,0.986878,58.309981,2,True,9
7,LightGBM_BAG_L2,-13792.005444,-26952.021161,root_mean_squared_error,12.445116,4.309834,294.183381,0.513953,0.316506,49.084109,2,True,10
8,LightGBM_BAG_L1,-15041.498027,-27049.776706,root_mean_squared_error,0.369585,0.277574,40.884871,0.369585,0.277574,40.884871,1,True,4
9,CatBoost_BAG_L1,-16560.513571,-25860.015841,root_mean_squared_error,0.145069,0.706186,123.729217,0.145069,0.706186,123.729217,1,True,6


In [6]:
# ทำนายผลบนข้อมูล test
predictions = predictor.predict(test_data_ag)

# สร้าง DataFrame สำหรับไฟล์ submission
submission = pd.DataFrame({
    'Id': test_ids,
    'SalePrice': predictions
})

# แสดงตัวอย่าง 5 แถวแรกของไฟล์ submission
print("ตัวอย่างไฟล์ Submission:")
submission.head()

# บันทึกเป็นไฟล์ CSV
submission.to_csv('submission_house_prices.csv', index=False)

print("\nสร้างไฟล์ submission_house_prices.csv เรียบร้อยแล้ว!")


ตัวอย่างไฟล์ Submission:

สร้างไฟล์ submission_house_prices.csv เรียบร้อยแล้ว!


In [ ]:
# submission = pd.DataFrame({
#     'Id': range(1, len(predictions) + 1),
#     'SalePrice': predictions
# })


In [7]:
submission

,Id,SalePrice
0,1461,120809.710938
1,1462,156084.625000
2,1463,185202.046875
3,1464,188728.687500
4,1465,190037.359375
...,...,...
1454,2915,85303.320312
1455,2916,86786.000000
1456,2917,160997.812500
1457,2918,117373.054688
